# Util Collab

In [65]:
# !python -m pip install tensorflow keras
# !python -m pip install scikit-learn
# !python -m pip install nltk

In [66]:
# !pip install tensorflow==2.8.2

In [67]:
# !pip install --upgrade tensorflow scikeras keras

# Import

In [68]:
import numpy as np
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Flatten,BatchNormalization
from keras import regularizers

import re
import os

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\doria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Coder le texte

## Récupération des textes et des années de publications (labels)

In [69]:
# Chemin du dossier contenant les textes
dossier_textes = 'CORPUS DATATION TER'

# Récupérer la liste des noms de fichiers dans le dossier
fichiers = os.listdir(dossier_textes)

# Trier les fichiers par ordre alphabétique
fichiers.sort()

# Initialiser une liste pour stocker les textes
books = []

# Initialiser une liste pour stocker les années extraites des noms de fichiers
y = []

# Parcourir les fichiers et lire leur contenu
for fichier in fichiers:
    chemin_fichier = os.path.join(dossier_textes, fichier)
    with open(chemin_fichier, 'r', encoding='utf-8') as f:

        texte = f.read()

        # Extraire l'année du nom de fichier
        annee = re.search(r'\((\d{4})\)', fichier)  # Utilisation d'une expression régulière pour trouver l'année entre parenthèses
        # Si une année est trouvée, alors nous pouvons étudier le livre, nous l'ajoutons ainsi que son label dans les listes associées
        if annee:
            books.append(texte)
            annee_int = int(annee.group(1)) # Récupérer l'année
            y.append(annee_int)  # Ajouter l'année extraite à la liste des labels

# Division de notre bdd en deux bdd, l'une pour l'entrainement et l'autre pour les tests
X_train, X_test, y_train, y_test = train_test_split(books, y, test_size=0.1, random_state=42)
X_train,X_val,y_train,y_val =  train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Afficher les annéees extraites
print(y_train)
#print(len(X_train[9]))

[1825, 2004, 2006, 1860, 2018, 1832, 1996, 1897, 1939, 1926, 1920, 1898, 1872, 1996, 1993, 1925, 1937, 2013, 1800, 1923, 1844, 1906, 2007, 1988, 1752, 1911, 2008, 1979, 1999, 1749, 1935, 1815, 1986, 1864, 1878, 1998, 1982, 1987, 1961, 1999, 1950, 1932, 2000, 1985, 1998, 1949, 1796, 1862, 1725, 2011, 1851, 1898, 1956, 2005, 1947, 1856, 1957, 1896, 1917, 1937, 1836, 1637, 2020, 1905, 1858, 1883, 1850, 1967, 1841, 1917, 1942, 1993, 1942, 1971, 1981, 1878, 1675, 1885, 1867, 1951, 1952, 1878, 1898, 1923, 1960, 1553, 1997, 1859, 1964, 1965, 1953, 2010, 1992, 1840, 1695, 1990, 1844, 1927, 1840, 1959, 1884, 1961, 1876, 2016, 2009, 1800, 1946, 1968, 1758, 1861, 1533, 1929, 1972, 1774, 2010, 1873, 1863, 1988, 1832, 1962, 1945, 1846, 1900, 1793, 1902, 1659, 2001, 2014, 2019, 1938, 1688, 1947, 1941, 1869, 1903, 1985, 1950, 1977, 2015, 1812, 1889, 1903, 2006, 1950, 1860, 1856, 2018, 1982, 1921, 1933, 1991, 1900, 1953, 1876, 1910, 2001, 1842, 1936, 1863, 1867, 1862, 1900, 2020, 1990, 1929, 1842, 196

## Tokenization + padding

In [70]:
# Paramètres du Tokenizer
max_len = 5000 # Longueur maximale des séquences

In [71]:
class Tokenizer2:
    def __init__(self):
        self.word_to_number = {}
        self.next_number = 0

    def tokenize(self, list_of_texts):
        total_texts = len(list_of_texts)
        token_lists = []
        for i, text in enumerate(list_of_texts):
            text = text.lower()
            tokens = word_tokenize(text)
            token_lists.append([])

            for token in tokens:
                if token not in self.word_to_number:
                    self.word_to_number[token] = self.next_number
                    self.next_number += 1

                token_lists[-1].append(self.word_to_number[token])

            percentage = (i + 1) / total_texts * 100
            print(f"Progress: {percentage:.2f}% complete")

        print("\nTokenization complete.")
        return token_lists, self.next_number
    
    def tokenizeNoAdd(self, list_of_texts):
        total_texts = len(list_of_texts)
        token_lists = []
        for i, text in enumerate(list_of_texts):
            text = text.lower()
            tokens = word_tokenize(text)
            token_lists.append([])

            for token in tokens:
                if token not in self.word_to_number:
                    continue

                token_lists[-1].append(self.word_to_number[token])

            percentage = (i + 1) / total_texts * 100
            print(f"Progress: {percentage:.2f}% complete")

        print("\nTokenization complete.")
        return token_lists, self.next_number

In [72]:
def diviser_liste(liste,labels, taille):
    resultats = []
    res_labels = []
    ind = 0
    # Parcours de chaque sous-liste dans la liste principale
    for sous_liste in liste:
        longueur = len(sous_liste)
        if(longueur<taille):
            ind+=1
            continue
        nbtour = round(longueur/taille)
        lastpos=0
        # Division de la sous-liste en morceaux de taille spécifiée
        for i in range(nbtour):
            if(lastpos + taille > longueur):
              lastpos -= (lastpos + taille)-longueur

            # Ajout du morceau à la liste de résultats
            resultat = sous_liste[lastpos:lastpos + taille]
            lastpos = lastpos + taille
            resultats.append(resultat)

            # A chaque fois que je divisie, jajoute le label au meme indice
            res_labels.append(labels[ind])
        ind+=1

    return resultats,res_labels

In [73]:
tokenizer = Tokenizer2()
# Transformation de chaque texte dans X_train en une séquence d'entiers
sequences, token_number = tokenizer.tokenize(X_train)

Progress: 0.12% complete
Progress: 0.24% complete
Progress: 0.35% complete
Progress: 0.47% complete
Progress: 0.59% complete
Progress: 0.71% complete
Progress: 0.82% complete
Progress: 0.94% complete
Progress: 1.06% complete
Progress: 1.18% complete
Progress: 1.29% complete
Progress: 1.41% complete
Progress: 1.53% complete
Progress: 1.65% complete
Progress: 1.76% complete
Progress: 1.88% complete
Progress: 2.00% complete
Progress: 2.12% complete
Progress: 2.23% complete
Progress: 2.35% complete
Progress: 2.47% complete
Progress: 2.59% complete
Progress: 2.70% complete
Progress: 2.82% complete
Progress: 2.94% complete
Progress: 3.06% complete
Progress: 3.17% complete
Progress: 3.29% complete
Progress: 3.41% complete
Progress: 3.53% complete
Progress: 3.64% complete
Progress: 3.76% complete
Progress: 3.88% complete
Progress: 4.00% complete
Progress: 4.11% complete
Progress: 4.23% complete
Progress: 4.35% complete
Progress: 4.47% complete
Progress: 4.58% complete
Progress: 4.70% complete


In [74]:
data,y_train = diviser_liste(sequences,y_train,max_len)

max_words = token_number
print(y_train)
# Convertir y_train en tableau numpy
y_train = np.array(y_train)
data = np.array(data)
print(len(data))
print("Nombre d'index dans le tokenizer:", max_words)

[1825, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 1860, 1860, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 1832, 1832, 1832, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1897, 1939, 1939, 1939, 1939, 1939, 1939, 1939, 1926, 1926, 1926, 1926, 1926, 1926, 1926, 1926, 1926, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1898, 1898, 1898, 1898, 1898, 1898, 1898, 1898, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1993, 1993, 199

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (12829,) + inhomogeneous part.

In [ ]:
len(sequences[2])

# Définition des architectures des modeles testés

In [ ]:
# # Construction du modèle
# # Pilee linéaire de couches. Dans ce modèle, les données passent à travers les couches dans l'ordre où elles ont été ajoutées.
# model = Sequential()
# # L'embedding est une représentation dense des mots (ou des tokens) dans un espace vectoriel.
# # L'objectif principal de l'embedding est de capturer les similarités sémantiques entre les mots en les plaçant dans un espace vectoriel où les mots ayant des significations similaires sont plus proches les uns des autres.
# model.add(Embedding(max_words, embedding_dim, input_length=max_len))

# # Couche de convolution à une dimension (1D) au modèle. Cette couche va utiliser 128 filtres de taille 10 avec une fonction d'activation ReLU (Rectified Linear Unit).
# # La convolution 1D est utilisée pour extraire des caractéristiques séquentielles des données.
# model.add(Conv1D(128,10, activation='relu'))

# # Le pooling global maximale extrait les caractéristiques les plus significatives de chaque canal de la sortie de la couche précédente, réduisant ainsi la dimensionnalité des données.
# model.add(GlobalMaxPooling1D())

# # Couche dense (entièrement connectée) au modèle avec 128 neurones et une fonction d'activation ReLU.
# model.add(Dense(128, activation='relu'))

# # Le dropout est une technique de régularisation qui désactive aléatoirement un pourcentage (dans ce cas, 50%) des neurones de la couche précédente pendant l'entraînement,
# # ce qui aide à prévenir le surapprentissage.
# model.add(Dropout(0.5))

# # Couche dense de sortie au modèle avec un seul neurone et une fonction d'activation linéaire.
# # Dans ce cas, la sortie est une seule valeur, qui est la date de publication prédite.
# model.add(Dense(1, activation='linear'))

# # La fonction ReLU est couramment utilisée dans les réseaux de neurones en raison de sa simplicité et de sa capacité à introduire une non-linéarité dans le modèle,
# # ce qui permet au réseau de capturer des motifs complexes dans les données.

# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# model.summary()

# # Entraînement du modèle
# model.fit(data, y_train, epochs=50, batch_size=32)
# #, validation_split=0.2
# # Après l'entraînement, vous pouvez utiliser le modèle pour prédire la date de publication d'un nouveau livre.


In [ ]:
embedding_dim = 8

def archi_3CONV32_F_DE_DR_DE():
  model = Sequential()

#inputdim = le nombre de voc, inputlength le nombre delement dans une liste
  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
  model.add(Conv1D(32, 5, activation='relu'))
  model.add(Conv1D(32, 5, activation='relu'))
  model.add(Conv1D(32, 5, activation='relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-6)))
  model.add(Dropout(0.5))
  model.add(Dense(1))

  return model

def archi_3CONV16_F_DE_DR_DE():
  model = Sequential()

  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-6)))
  model.add(Dropout(0.5))
  model.add(Dense(1))
  return model

def archi_3CONV64_F_DE_DR_DE():
  model = Sequential()

  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
  model.add(Conv1D(64, 5, activation='relu'))
  model.add(Conv1D(64, 5, activation='relu'))
  model.add(Conv1D(64, 5, activation='relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-6)))
  model.add(Dropout(0.5))
  model.add(Dense(1))

  return model

def archi_3CONV64_32_16_F_DE_DR_DE():
  model = Sequential()

  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
  model.add(Conv1D(64, 5, activation='relu'))
  model.add(Conv1D(32, 5, activation='relu'))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-6)))
  model.add(Dropout(0.5))
  model.add(Dense(1))

  return model

# Prétraitement données de test

In [ ]:
# Transformation de chaque texte dans X_train en une séquence d'entiers
sequences_test, _ = tokenizer.tokenizeNoAdd(X_test)

data_test,y_test = diviser_liste(sequences_test,y_test,max_len)

# Convertir y_train en tableau numpy
y_test = np.array(y_test)
data_test = np.array(data_test)

# Transformation de chaque texte dans X_train en une séquence d'entiers
sequences_val, _ = tokenizer.tokenizeNoAdd(X_val)

data_val,y_val = diviser_liste(sequences_val,y_val,max_len)

# Convertir y_train en tableau numpy
y_val = np.array(y_val)
data_val = np.array(data_val)

# Entrainement des models

## Trouver le meilleur

In [ ]:
# # Définir la liste des modèles à évaluer
# models = [archi_3CONV32_F_DE_DR_DE(), archi_3CONV16_F_DE_DR_DE(), archi_3CONV64_F_DE_DR_DE(), archi_3CONV64_32_16_F_DE_DR_DE()]

# best_model = None
# best_val_loss = float('inf')
# best_val_mae = float('inf')

# # Entraîner et évaluer chaque modèle
# for model in models:
#     model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
#     model.fit(data, y_train, epochs=20, batch_size=64, validation_data=(data_val, y_val))

#     # Évaluer sur l'ensemble de test
#     val_loss, val_mae = model.evaluate(data_test, y_test)

#     # Mettre à jour le meilleur modèle si nécessaire
#     if val_mae < best_val_mae:
#         best_val_loss = val_loss
#         best_val_mae = val_mae
#         best_model = model

# # Afficher les performances du meilleur modèle
# print("Le meilleur modèle est:")
# best_model.summary()
# print("Meilleure valeur de perte (loss) sur l'ensemble de test:", best_val_loss)
# print("Meilleure MAE (Mean Absolute Error) sur l'ensemble de test:", best_val_mae)


In [ ]:
# model = best_model

## Ou en entrainer un seul

In [ ]:
model = archi_3CONV32_F_DE_DR_DE()

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

# Entraînement du modèle
history = model.fit(data, y_train, epochs=20, batch_size=64, validation_data=(data_val, y_val))
history.history

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 5000, 8)           723456    
                                                                 
 conv1d_39 (Conv1D)          (None, 4996, 32)          1312      
                                                                 
 conv1d_40 (Conv1D)          (None, 4992, 32)          5152      
                                                                 
 conv1d_41 (Conv1D)          (None, 4988, 32)          5152      
                                                                 
 flatten_13 (Flatten)        (None, 159616)            0         
                                                                 
 dense_26 (Dense)            (None, 256)               40861952  
                                                                 
 dropout_13 (Dropout)        (None, 256)             

{'loss': [1839327.0,
  144614.078125,
  49191.734375,
  43837.64453125,
  37158.546875,
  39958.67578125,
  35709.4765625,
  35416.83984375,
  35617.2890625,
  33846.63671875,
  34626.48828125,
  34706.16015625,
  32323.94140625,
  31377.197265625,
  30402.994140625,
  34841.64453125,
  31685.34765625,
  32897.0234375,
  31140.029296875,
  31445.72265625],
 'mae': [1198.0447998046875,
  318.7552795410156,
  179.47537231445312,
  167.85447692871094,
  155.38389587402344,
  159.56683349609375,
  149.3181915283203,
  151.37290954589844,
  149.90672302246094,
  148.23854064941406,
  148.01004028320312,
  148.3927001953125,
  141.95803833007812,
  139.7604522705078,
  140.83030700683594,
  148.97463989257812,
  143.5814208984375,
  145.6296844482422,
  139.43324279785156,
  143.06935119628906],
 'val_loss': [20698.517578125,
  3286.456787109375,
  15619.025390625,
  6626.19287109375,
  3098.871826171875,
  2888.220703125,
  1651.26953125,
  1582.8306884765625,
  1941.5296630859375,
  664.86

In [ ]:
# Evaluate the model on the test data
val_loss, val_mae = model.evaluate(data_test, y_test)

# Print the validation loss and validation mean absolute error
print("Validation Loss:", val_loss)
print("Validation MAE:", val_mae)

7/7 [==============================] - 0s 23ms/step - loss: 1301.2847 - mae: 31.7301
Validation Loss: 1301.28466796875
Validation MAE: 31.730117797851562
